<h1><center>Continuous Integration for Data Scientsts</center></h1>
<h2><center>by Jaidev Deshpande</center></h2>
<h3><center>Data Scientist @ Cube26 Software Pvt Ltd</center></h3>

<div style="text-align: center">
<div id="social">
<div id="social_twitter">
    <a href="http://twitter.com/jaidevd"><img src="images/twitter-128.png" width="32" height="32">
    </a>
</div>
<div id="social_medium">
    <a href="http://medium.com/@jaidevd"><img src="images/medium.ico" width="32" height="32"></a>
</div>
<div id="social_github">
    <a href="http://github.com/jaidevd"><img src="images/mark-github-128.png" width="32" height="32"></a>
</div>
</div>
</div>

<div style="text-align: center"><font size="4"><strong>@jaidevd</strong></font></div>

# About me:

* Data Scientist @Cube26 Software Pvt Ltd
* Regular contributor to SciPy Stack
* Helped develop the Canopy Data Analysis Platform

# From Peter Norvig's Q&A session on Quora:
> I think that it will be important for machine learning experts and software engineers to come together to develop best practices for software development of machine learning systems. Currently we have a software testing regime where you define unit tests... We will need new testing processes that involve running experiments, analyzing the results... This is a great area for software engineers and machine learning people to work together to build something new and better.

Recently Peter Norvig was asked in a Q&A what he thinks about the place of software developers in machine learning research. And this is his answer. "It's important for machine learning experts and software engineers to come together to develop best practices for the development of ML systems. We need new testing processes that involve running experiments and analyzing results" and that "this is a great area for software developers and ML expers to work together". The interesting thing here is that Norvig is talking about testing not as quality control, but as a way of evaluating machine learning models. So this broadly the idea is what I'm going to speak about, today. When I say continuous integration, I don't literally mean CI. CI is a placeholder term for CI systems, devops, build, test and QA systems - and also the culture and habits that come with it.

# Example: How we started building ReosMessage

<ul>
    <li><h3>Classification of SMS into personal, transactional and spam</h3></li>
    <li><h3>Get a dump of a table from postgres</h3></li>
    <li><h3>Use simple Python scripts and some pandas magic to clean the data</h3></li>
    <li><h3>Use regular expressions to label the data</h3></li>
    <li><h3>Train sklearn estimators on the labeled data</h3></li>
    <li><h3>Crowdsource the evaluation of the predictions</h3></li>
    <li><h3>Dump the model coefficients to JSON</h3></li>
    <li><h3>Hand over the JSON to Android developers</h3></li>
</ul>

I'll start with an example, and at the end of the talk I'll come back to how it evolved. At Cube26, we recently released a suite of apps called ReosApps. One of them was the ReosMessage app - which classifies your messages into personal, transactional or notification messages and spam. So to build and deploy the classifier, all we had initially was a massive Python script - with a single input which was a CSV file. This file was just a dump of a table from a postgres database. We then would run some simple pandas functions to clean the data of unicode, emojis - anything else that was irrelevant. We didn't have any labels, so we used regular expressions to label a subset of the dataset into those three classes. On this data we trained some scikit-learn models. We used to send the model results to everyone in the office. They would all quickly glance over it let us know if they were any glaring classification errors. When we were somewhat satisfied with the results, the model coefficients were dumped into the JSON and were sent to the Android developers who could use them in their port of the classifier. It's easy to see how this could not have scaled. It's extremely monolithic, it's quite redundant in places, it wasn't modular at all, and also it was becoming increasingly difficult to debug. So I'll come back to this example at the end of the talk and I'll tell you what we did to fix these things.

# Typical Data Processing Pipeline
![](images/flowchart.png)

So this here is a very broadly representative, a very general data processing pipeline. That box you see to the left is known as ETL part of the pipeline. ETL stands for extract, transform and load. You start with a source datastore, and end up in a sink datastore. The ETL part of the process is the most data-intensive one - by that I don't mean that others are not, it's just that this is the part where you treat your data for what it is - raw data. After it's left this box, it's no longer raw data, it's features, it's model coefficients, it's statistics, and so on. And it is in this box that your developer side has to be the most creative, as against your data scientist side, which has to show off outside the box. So after you've extracted, transformed and loaded your data, you train some model on it, and you do some validation which might allow you to do better model selection. Now, these steps - training, validation and model selection - are not necessarily well separated in time. You could be cross-validating one model while you are already training another model, and a third one could actually be in production that is writing the output to whatever your sink is. But the thing is that we tend to think about a pipeline as something sequential. You can't think if this as a pipeline. How do you bend the spoon? Think that there is no spoon. Think that there is no such pipeline, and each of these blocks are independent systems - these are microservices - which only happen to be loosely coupled.

# Using Data Abstractions
## And Data Ingest as an abstraction
![](images/wc.png)

This is a word cloud of the text of all the proposals made to PyCon. Now humour me a bit over here. As a species, I think we are spending an enormous amount of time and money _simply_ beating data into shape. That's what almost all our conferences are about. Although I cannot justify this, I have a feeling that that is really overkill. I feel like dealing with data should be easier.

I've met people who very sincerely believe that if they can get a lot of data accumulated in one place, it will automatically start becoming sentient. And this is for no fault of their own. This is clearly ridiculous. Data may not be sentient, but there certainly are some advantages to thinking about it as a living entity - in that it needs to be grown and cultivated before it is harvested.

# Using Data Abstractions
<div id="data_min_farm">
<div id="data_min_farm_min">
    <img src="images/mining.jpg">
</div>
<h3><center>vs</center></h3>
<div id="data_min_farm_farm">
    <img src="images/farming.jpg">
</div>
</div>


So instead of data mining, let's start speaking in terms of data farming. And here are a few ways of doing that.

# Using Data Abstractions
## And Data Ingest as an abstraction

<ul>
    <li><h3>Raw data is an integral part of ETL and therefore of your software</h3></li>
    <li><h3>Working off local flatflies is <em>bad</em>!</h3></li>
    <li><h3>Learn to work from remote storage to remote storage. Use the "cloud".</h3></li>
    <li><h3>What about experimental / exploratory work? Use things like sqlite!</h3></li>
    <li><h3>Only use local files when:</h3></li>
    <ul>
        <li><h3>doing EDA or any interactive studies.</h3></li>
        <li><h3>debugging a larger application.</h3></li>
        <li><h3>prototyping (resist the temptation to deploy).</h3></li>
    </ul>
</ul>

Abstract away almost everything that has to do with ETL. We, as a community, the data science community, haven't paid much attention to it. Machine learning systems have grown super sophisticated while data management systems (that these same machine learning systems use), have trudged behind. Let's stop thinking of raw data as an inanimate entity that you only have to dig through before you can get the gold. The digging itself is part of your software, and is therefore subject to pretty much the same dangers as any other kind of software development. So build abstraction layers, services and all kinds of tooling that you would need around ETL. Now this might sound very obvious, but you'd be surprised at how infrequently we practice this. For example, almost all development data scientists do is based off local flatfiles - most of it actually happens within the ipython notebook. The excuse is that they're just building prototypes and it's not really their job to have to deploy their models on larger systems. But we know how fast the boundaries between data scientists and other kinds of developers are thinning. In that light, we have to learn to work with larger more intergated data sources. Even if you're just building a prototype, this is still not every healthy because you have no idea how long the prototyping is going to take, or how many intermediate files you might end up producing. So, at least as a favour to people who are deploying your work to larger systems, learn to use larger integrated systems. Even if you're using the Iris dataset, try to use sqlite instead of CSV. The cloud is your friend. The sooner you become comfortable with remote or cloud based distributed storages, the faster you can deploy your apps.

# Data Ingest Libraries

<ul>
    <li><h3>sputnik: managing data packages and ML models</h3></li>
    <li><h3>datreant - Using the local filesystem</h3></li>
    <ul>
        <li><h3>heterogenous data</h3></li>
        <li><h3>many parameters, many files - lot of mess</h3></li>
        <li><h3>datreant exploits this mess as flexibility, discoverability</h3></li>
    </ul>
    <li><h3>conduit: data exchange for HPC</h3></li>
    <ul>
        <li><h3>in-memory exchange & file-based exchange of data</h3></li>
        <li><h3>JSON schema to describe and annotate data</h3></li>
    </ul>
    <li><h3>pysemantic: automate the validation and cleaning of dataset</h3></li>
</ul>

And here are a few libraries that can help you do that. So Sputnik is a library that manages data packages. You take a directory with your data in it, add a json that describes that directory and Sputnik will make a tarball out of it, allowing you to deal with the whole thing in a very atomic manner. Datreant is another data management library that leverages the local filesystem itself. It's common to have a very messy filesystem, but datreant will exploit this mess to provide some sort of discoverability into your data. You can think of it as a tradeoff between the simplicity of rigid file structure and the heterogenous data storage functionality that your filesystem can provide. Then there is conduit - which is used for managing data for HPC simulations. It is essentially a data model for describing hierarchical datasets. So it represents your data as a graph - and this graph integrates seamlessly with the in-memory data, serialized on-disk data, etc. Finally there is PySemantic by yours truly - which automates all the pandas code you would write for validating and cleaning your dataset.

# A Note about the AutoML project
![](images/automl.png)
<h4><i>"If machine learning is so ubiquitous, one should just be able to use it without understanding libraries."</i></h4>
<h4>- Andreas Mueller @ SciPy US 2016</h4>

<li><h3>sklearn philosophy: explicit is better than implicit. DevOps philosophy - Just build and run stuff!</h3>


So that was all about data ingest. Now let's talk about automating machine learning. This is a very widely studied problem, and one of its most popular forms is the AutoML problem - also known as the CASH problem. CASH stands for Combined Algorithm Selection and Hyperparameter Optimization. The motivation for formally studying automation of machine learning is simple - no algorithm works best on all datasets, not even neural networks - and most algorithms are highly sensitive to hyperparameters. This here is a concise flowchart of the AutoML framework. The _xTrain, xTest, yTrain_ variables are as usual, L is the loss. The interesting thing here is b which denotes your budget - which could be time or processing power. The ML framework box here is as usual. There's a data preprocessor, which feeds into a feature extractor and then finally a classifier. The _Bayesian optimizer_ block is a process that fits a model between hyperparameters and performance. Actually take a moment to think about that. You want to automate the training of a model, and for that you're fitting a model which given another model and its hyperparameters, predicts the performance. The meta-learning block here also learns the performance, but the input to that is the dataset. So given a dataset, it predicts performance, you know, like a domain expert. So when we say that decision trees are likely to perform well for categorical variables - that is the sort of thing that the meta-learning block learns. Finally, we build ensembles from the different models that have been performing well. Thankfully, there's a convenient sklearn implementation of this - called auto-sklearn. This is what Andreas Mueller spoke about in his talk at SciPy this year. The idea is that if machine learning is so ubuquitous then one should be able to just use it without understanding the details. Now, if you think about it, this is exactly how sysadmins think.

Now allow me to digress a bit. About two years ago I used to work on the Enthought Python Distribution, or Canopy. I was briefly a part of the buildsystem team that was responsible for making Python packages that would work across many platforms and architectures reliably. One rule of thumb that the buildsystem team had was that _upstream is always broken_. If you built a Python package as-is from its source code repository - it was very likely that it would be broken for some platform. So we used to write our own patches to fix that. We had patches for almost every project which we would apply and then build the packages before distributing them. Sklearn is one precious few projects which did not require a patch. It's a wonderful library that you really should study. Like requests, its one of those projects that are an example of what good Python development should be like. So it's very natural that sklearn follows the Zen of Python. An sklearn developer would say that explicit is better than implicit. But a sysadmin would say, _is it really_? For sysadmins, things are so automated that its impossible to make them reasonably explicit. In fact, most sysadmins are reluctant even to update libraries lest it break something else. This is one of the reasons why enterprise Linux distros have outdated libraries. They just care about building and running stuff - which is another reason why projects like AutoML are useful. They allow you to run _a lot_ of models easily and in a hands-off manner.

# A Note about the AutoML project
<ul>
    <li><h3>Formalize the search space mindfully - discrete or continuous? Conditional parameters?</h3></li>
    <li><h3>Automate:</h3></li>
    <ul>
        <li><h3>Choice of classification/regression algorithm</h3></li>
        <li><h3>feature extraction & preprocessing</h3></li>
        <li><h3>hyperparameter tuning</h3></li>
    </ul>
    <li><h3>Randomized & Grid Search are both embarassingly parallel</h3></li>
</ul>

But then, with great power comes great responsibilty. Automating machine learning is expensive. So formalize the search space mindfully. Think about which parameters are continuous or discrete. Especially in sklearn there are a _combinations_ of hyperparameters that are not compatible with themselves. When you are iterating though a grid, the  gridsearch reaches an incompatible set of hyperparamters, and fails. So unless you have been logging everything, you have to start all over again. Normally the habit is to stuff everything you can into the grid - but much of the grid can be restricted before you even start, if you think it through properly.

So out of these three things, the choice of the estimator and the hyperparamter tuning can be automated with things like GridSearch and auto-sklearn, but there's no easy automation strategy for feature extraction and preprocessing that I know of. That is still something that has to be done by hand a few times before you can reach a reasonable choice.

# Automating Model Selection
## Automating Cross Validation and Hyperparameter Tuning

```python
class CrossValidationTask(luigi.Task):
    
    estimator = luigi.Parameter() # or luigi.Target
    
    def run(self):
        # Run CV loop
        # Export metrics for each iteration


class GridSearchTask(luigi.Task):
    
    grid = luigi.Parameter() # or Target
    estimator = luigi.Parameter() # or Target
    ...
    
    def run(self):
        X, y = self.input()
        clf = GridSearchCV(self.estimator, param_grid=self.grid, ...)
        clf.fit(X, y)
        ...
        joblib.dump(clf.best_estimator_, self.output())
```

This is a sample of how you would do all of that with luigi. Luigi is a open-source pipelining framework by spotify. Think if it as a buildsystem like make, except its a lot more flexible and distributed. You can define these as tasks in your pipeline. Luigi tasks can be parameterized - just like Jenkins builds are parameterized - by any Python object. As long as it's hashable, of course. So you can use your estimator itself as the parameter, and run a cross validation loop, and put the results on a dashboard, so you can visualize how well a model is performing at any given time. For the grid search, you can keep producing the best estimator for each run as a build artifact.

# Data and Model _Quality_

## (Tools from psychometrics for) data quality evaluation
### - Katie Malone @ SciPy US 2016

<ul>
    <li><h3>Predictive modeling != building a model</h3></li>
    <li><h3>Iterative model selection involves going all the way back to data quality (not simply changing the pipeline)</h3></li>
    <li><h3>Develop the same intuition for your data as that for your model</h3></li>
    <ul>
        <li><h3>complex model + mediocre dataset = fair predictive accuracy</h3></li>
        <li><h3>simple model + great dataset = high predictive accuracy</h3></li>
    </ul>
</ul>

So that was all about hacking and creating models. Now let's talk about inference and quality of the models. There was an interesting talk at SciPy this year by Katie Malone about using tools from pyschometrics for data quality evaluation. Data quality is again not something we think of too often. We tend to be so preoccupied with model performance that we don't think of data quality as a measurable entity. Do look up the talk. In summary the talk says that predictive modeling is not the same as building a model. It needs to be more comprehensive than that, something with _insight_. So, iterating over different models shouldn't be limited to simply changing feature extraction or hyperparameter selection. You need to go all the way back to your data. This ties into what I meant when I spoke of data _farming_ instead of data mining. Have the same intuition for your data as you have for your model. Specifically, that means asking questions like what dataset would give me better performance for a fixed model. So essentially turn around your intuition about predictive modeling. You'll quite likely find that simpler models perform much better on good datasets, but if your data isn't good enough, even complex models won't do well. And there's of course the overhead of building and maintaining complex models that you may want to avoid.

# Data and model _Quality_
## Communicating model results
### - Bargava Subramanian @ SciPy US 2016

<ul>
    <li><h3>Performance metrics aren't as important as insights</h3></li>
    <li><h3>Modularize and abstract away:</h3></li>
    <ol>
        <li><h3>raw data behaviour</h3></li>
        <li><h3>visualization</h3></li>
        <li><h3>learning and metrics</h3></li>
    </ol>
</ul>

Speaking of inference and insights - there was another interesting talk at SciPy by Bargava Subramanian on communicating model results. He points out that performance metrics are not as important as model insights. In fact, most people can't even understand what precision and recall scores are. So the inference has to focus on insights about your data. As an example, I was trying to model the talk selection process here at PyCon with a decision tree. The model was doing fairly well, but instead of pushing for an extra percent in classification accuracy, its better if I spend time visualizing the tree and its decision making process - its better to see how well the tree reflects our talk selection process rather than simply being a prediction model that tells you whether a talk is likely to be selected. I might do a lightning talk about this later during the conference. And in that spirit, the software should be modular with respect to,
first of all the original data, then the interpretation of the model and then _finally_ the metrics - you know, as a formality. Don't let the metrics take undue precedence over the data or the learning process.

# Visualizing (Data & ML performance)

<ul>
        <li><h3>Bokeh server for dashboards</h3></li>
        <li><h3>Chaco / Traits-based visualizations - for interative exploration</h3></li>
        <li><h3>Use libs like Seaborn for stats - resist the temptation to write them yourself</h3></li>
    
</ul>

These are a few things that could help you with the visualization beyond matplotlib. Bokeh is wonderful for dynamic plotting. You don't have to know HTML or js, and it runs as a server and you can keep sending it data to have it blotted dynamically. Instant dashboards. Chaco is another library that provides a lot more interactivity than matplotlib, but is somewhat more difficult to hack - and its not easy to get it working in the browser. There are many other libraries like seaborn, geopandas, networkx that have domain specific data visualization. Use these domain specific visualization tools, and try not to write your own.

# Exposing Trained Models

<ul>
        <li><h3>Simple serialization methods</h3></li>
        <li><h3>sklearn-compiledtrees</h3></li>
        <li><h3>Don't use nonlinear models where linear models will do</h3></li>
    
</ul>

Finally we come to deployment. The easiest way to deploy models is to use simple serialization methods like pickle or json - sklearn's joblib will do great - and then writing wrappers around the serialized models on the client side - especially if its not a Python client. sklearn-compiledtrees is a project that compiles tree based models from sklearn into object code, which can then be read by the client. Now you _could_ port almost any machine learning algorithm to any platform, but if it's a linear model, then the advantage you have is that the client needs to compute only a dot product. So evaluate linear models first, and if you can't work with them, make absolutely sure that you can't.

# Exposing Trained Models - The Serverless Paradigm

> Most MLaaS products don’t allow much personalisation.
This is why the Data Science community is still “stuck” to designing ad-hoc models with lower-level tools and frameworks. Which is great. Except that most of the time these ad-hoc models are developed by data scientists and tend to get stuck at the prototyping level. The outcome of this process will require a huge effort by some other developer or engineer, who probably can’t speak the data science language.

> What if prototypes developed by data scientists were production-ready without any further effort?

\- Alex Casalboni
(PyCon Italia 2015)



An interesting deployment solution that has emerged recently is the serverless paradigm. Alex Casalboni is a developer at Cloudacademy where they are pioneering the serverless paradigm specially in the Python/Scikit-learn ecosystem. He says that the data science community is stuck at the level of designing ad-hoc models - which are mostly just prototype with very thin layers of customization. And they cannot be embedded or extended by other developers very easily. So the serverless paradigm is one of the ways in which you can make your prototypes production-ready without too much effort.

# Exposing Trained Models - The Serverless Paradigm
<ul>
        <li><h3>Simple serialization methods - are not infrastructure aware</h3></li>
        <li><h3>HTTP API? Authentication?</h3></li>
        <li><h3>Deployment Strategies:</h3></li>
        <ol>
        <li><h4>Simple flask wrappers - what about model updates?</h4>
        <li><h4>Backend scaling? No elasticity!</h4>
        <li><h4>AWS Lambda / Amazon API Gateway - Just use a set of functions that are automatically deployed and scaled</h4>
        </ol>
    
</ul>

Now even if we want to keep things simple with simple serialization - its a lot of effort to scale it, and to make it distributed. And you will also have to put a lot of thought into how you expose the API and how can authenticate it. One of the easiest things you could do it write simple flask or django wrappers around your model and make the estimator methods available through HTTP requests - but even that doesn't provide enough elasticity, and scaling is still non trivial. AWS Lambda is a service that allows you to just deploy scripts onto AWS and it takes care of the scaling automatically. A combination of this and the Amazon API Gateway is all you need for deploying and scaling your scikit-learn code. But unsurprisingly, that's more on the expensive slide.

# Recap
<ol>
        <li><h3>Abstract away all ETL and related code.</h3></li>
        <li><h3>Automating ML != brute force</h3></li>
        <li><h3>Insights >> performance metrics</h3></li>
        <li><h3>Modularity on all levels</h3></li>
    
</ol>

So, to recap, if there were a few key points that I'd like you to take away, then:

*number one* would be to abstract away your ETL code
*second and perhaps most important* searching for a good machine learning algorithm and tuning its parameters should be done in a much more thoughtful manner than just brute force
and finally, insights are more important than metrics. If you can't explain your model, you need to revaluate.

And of course, all of these systems ought to be modular. Use microservices wherever possible.

# Example: How we built & scaled ReosMessage

<ul>
    <li><h3><strike>Get a dump of a table from postgres</strike></h3></li>
    <li><h3><strike>Use simple Python scripts and some pandas magic to clean the data</strike></h3></li>
    <li><h3>Spark streaming API connected to Kafka consumers</h3></li>
    <li><h3>Use <strike>regular expressions</strike> user feedback to label the data</h3></li>
    <li><h3>Use Luigi to:</h3></li>
        <ul>
        <li><h3>Continuously run grid search and cross validation benchmarks</h3></li>
        <li><h3>Train sklearn estimators on the labeled data</h3></li>
        <li><h3>Dump the model coefficients to JSON</h3></li>
        <li><h3>Hand over the JSON to Android developers</h3></li>
        </ul>
    <li><h3>Use Jenkins to drive everything</h3></li>
</ul>

So coming back to the example from the beginning. Our postgres database was being populated by kafka consumers which in turn were reading data from the devices on which our apps are running. So instead of taking dumps from postgres, as we were earlier doing, we set up a streaming application in spark that served as a data ingest system. Spark itself has great data cleaning functionality - so we could manage to get the streaming application to absorb even the preprocessing layer we earlier had. The data then goes to a set of luigi tasks which are running parallely to produce newer benchmarks on the classification problems and ultimately the export the model configuration which ultimately gets deployed on the apps. And all of this is triggered by Jenkins - primarily, builds are triggered by cron jobs and whenever commits are made to the staging branches in git.

![](images/no_data.jpg)

Thanks for your time. Thank you very much.